# House Price Predictor Using Linear Regression

Data Source:- Kaggel

The Data is given two diffrent files Training and testing

# The steps taken to create model:
    1.Data Cleaning :
        It includes missing data treatment ,excluding unwanted data
    2.Co-relation check and selecting feature with high corelation
    3.Checking Skewness in data and treating it with log function
    4.Standardisation of data :
        For Categorical data One Hot Encoding is used
        For Continuous data StandardScalar is used
    5.Feature Engineering:
        It includes selection of more relevant features for your machine learning model on basis of P-value
        To select feature with 0.05 or less than ,I created OLS Model.
    6.Different Regression models used and Regularised its hyperparameters using GridSearchCV and RandomiseSearchCV to reduce Error.
    7.Performance metrics used here is Mean Absolute Error.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import mean_absolute_error


In [2]:
from warnings import filterwarnings
filterwarnings('ignore')

In [3]:
df=pd.read_csv('C:/Users/cmscomputers/Desktop/datasets/House_training_set.csv')

In [4]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df.isna().sum().sort_values(ascending=False)

PoolQC         1453
MiscFeature    1406
Alley          1369
Fence          1179
FireplaceQu     690
               ... 
ExterQual         0
Exterior2nd       0
Exterior1st       0
RoofMatl          0
SalePrice         0
Length: 81, dtype: int64

In [6]:
df=df.drop(labels=['PoolQC','MiscFeature','Alley','Fence'],axis=1)                  #dropping features with most of null values

In [7]:
df.isna().sum()

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 77, dtype: int64

In [8]:
for i in df:
    if(df[i].dtypes=='object'):
        c=df[i].mode()[0]
        df[i]=df[i].fillna(c)         #this iteration replaces missing values from categorical data
    else:
        c=df[i].mean()
        df[i]=df[i].fillna(c) 

In [9]:
df.isna().sum().sum()

0

In [10]:
df.skew().sort_values(ascending=False)

MiscVal          24.476794
PoolArea         14.828374
LotArea          12.207688
3SsnPorch        10.304342
LowQualFinSF      9.011341
KitchenAbvGr      4.488397
BsmtFinSF2        4.255261
ScreenPorch       4.122214
BsmtHalfBath      4.103403
EnclosedPorch     3.089872
MasVnrArea        2.676412
LotFrontage       2.384950
OpenPorchSF       2.364342
SalePrice         1.882876
BsmtFinSF1        1.685503
WoodDeckSF        1.541376
TotalBsmtSF       1.524255
MSSubClass        1.407657
1stFlrSF          1.376757
GrLivArea         1.366560
BsmtUnfSF         0.920268
2ndFlrSF          0.813030
OverallCond       0.693067
TotRmsAbvGrd      0.676341
HalfBath          0.675897
Fireplaces        0.649565
BsmtFullBath      0.596067
OverallQual       0.216944
MoSold            0.212053
BedroomAbvGr      0.211790
GarageArea        0.179981
YrSold            0.096269
FullBath          0.036562
Id                0.000000
GarageCars       -0.342549
YearRemodAdd     -0.503562
YearBuilt        -0.613461
G

In [11]:
def skew_rem(df,col):
    q = []
    for i in df[col]:
        if(i != 0):
            q.append(np.log(i))
        else:
            q.append(i)
    df[col] = q                #created a function to perform log function on skewed feature excluding 0 values ,
                                 # because log of 0 is infinite           

In [12]:
col=['MiscVal','PoolArea','LotFrontage','LotArea',
 'MasVnrArea','OpenPorchSF','3SsnPorch','LowQualFinSF','KitchenAbvGr','BsmtFinSF2',
'ScreenPorch',
'BsmtHalfBath']        #selected columns with skew more than 2

In [13]:
for i in col:
    skew_rem(df,i) 

In [14]:
D=df.corr()['SalePrice'].sort_values(ascending=False)
D

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.470177
Fireplaces       0.466929
OpenPorchSF      0.430243
MasVnrArea       0.411873
LotArea          0.388520
BsmtFinSF1       0.386420
LotFrontage      0.331479
WoodDeckSF       0.324413
2ndFlrSF         0.319334
HalfBath         0.284108
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.093965
PoolArea         0.093700
3SsnPorch        0.047339
MoSold           0.046432
BsmtHalfBath    -0.009360
Id              -0.021917
YrSold          -0.028923
LowQualFinSF    -0.044340
BsmtFinSF2      -0.048248
MiscVal         -0.063477
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.139670
Name: SalePr

In [15]:
D=pd.DataFrame(D)
D.columns=['corr']

In [16]:
E=D[(D['corr']<0.15)].index
E

Index(['ScreenPorch', 'PoolArea', '3SsnPorch', 'MoSold', 'BsmtHalfBath', 'Id',
       'YrSold', 'LowQualFinSF', 'BsmtFinSF2', 'MiscVal', 'OverallCond',
       'MSSubClass', 'EnclosedPorch', 'KitchenAbvGr'],
      dtype='object')

In [17]:
df=df.drop(labels=E,axis=1)
df.head()
#dropped features with less than 0.15 

,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,SaleType,SaleCondition,SalePrice
0,RL,4.174387,9.041922,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,2,548,TA,TA,Y,0,4.110874,WD,Normal,208500
1,RL,4.382027,9.169518,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,2,460,TA,TA,Y,298,0.000000,WD,Normal,181500
2,RL,4.219508,9.328123,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,2,608,TA,TA,Y,0,3.737670,WD,Normal,223500
3,RL,4.094345,9.164296,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,3,642,TA,TA,Y,0,3.555348,WD,Abnorml,140000
4,RL,4.430817,9.565214,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,3,836,TA,TA,Y,192,4.430817,WD,Normal,250000


In [18]:
cat=[]
con=[]
for i in df.columns:
    if(df[i].dtype=='object'):
        cat.append(i)
    else:
        con.append(i)

In [19]:
def ANOVA(df,cat,con):
    import pandas as pd
    from statsmodels.api import OLS
    from statsmodels.formula.api import ols
    rel = con + " ~ " + cat
    model = ols(rel,df).fit()
    from statsmodels.stats.anova import anova_lm
    anova_results = anova_lm(model)
    Q = pd.DataFrame(anova_results)
    a = Q['PR(>F)'][cat]
    return round(a,3)

In [20]:
imp_cols = []
for i in cat:
    q = ANOVA(df,i,'SalePrice')
    print("-------------")
    print("Attrition vs",i)
    print("Pval: ",q)
    if(q < 0.05):
        imp_cols.append(i)

-------------
Attrition vs MSZoning
Pval:  0.0
-------------
Attrition vs Street
Pval:  0.117
-------------
Attrition vs LotShape
Pval:  0.0
-------------
Attrition vs LandContour
Pval:  0.0
-------------
Attrition vs Utilities
Pval:  0.585
-------------
Attrition vs LotConfig
Pval:  0.0
-------------
Attrition vs LandSlope
Pval:  0.141
-------------
Attrition vs Neighborhood
Pval:  0.0
-------------
Attrition vs Condition1
Pval:  0.0
-------------
Attrition vs Condition2
Pval:  0.043
-------------
Attrition vs BldgType
Pval:  0.0
-------------
Attrition vs HouseStyle
Pval:  0.0
-------------
Attrition vs RoofStyle
Pval:  0.0
-------------
Attrition vs RoofMatl
Pval:  0.0
-------------
Attrition vs Exterior1st
Pval:  0.0
-------------
Attrition vs Exterior2nd
Pval:  0.0
-------------
Attrition vs MasVnrType
Pval:  0.0
-------------
Attrition vs ExterQual
Pval:  0.0
-------------
Attrition vs ExterCond
Pval:  0.0
-------------
Attrition vs Foundation
Pval:  0.0
-------------
Attrition v

In [21]:
len(imp_cols)

36

In [22]:
len(cat)

39

In [23]:
p=con+imp_cols

In [24]:
df.shape

(1460, 63)

In [25]:
len(p)

60

In [26]:
df=df[p]

In [27]:
df.shape

(1460, 60)

In [28]:
cat=[]
con=[]
for i in df.columns:
    if(df[i].dtype=='object'):
        cat.append(i)
    else:
        con.append(i)

Univariate

for i in df[con]:
        sb.boxplot(df[i],color ="r")
        plt.show()
        sb.distplot(df[i],color ="b")
        plt.show()

Bivariate

for i in df[cat]:
        sb.countplot(df[i],palette = "husl")
        plt.show()

# Data Standedization

In [29]:
train_cat=pd.get_dummies(df[cat])
train_cat.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [30]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
train_con=pd.DataFrame(ss.fit_transform(df[con]),columns=con)
train_con.head()

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,SalePrice
0,-0.084142,-0.133231,0.651479,1.050994,0.878668,1.193663,0.575425,-0.944591,-0.459303,-0.793434,...,1.227585,0.163779,0.912210,-0.951226,1.021157,0.311725,0.351000,-0.752176,0.845412,0.347273
1,0.563447,0.113442,-0.071836,0.156734,-0.429577,-0.815039,1.171992,-0.641228,0.466465,0.257140,...,-0.761621,0.163779,-0.318683,0.600495,-0.104483,0.311725,-0.060731,1.626195,-1.071354,0.007288
2,0.056580,0.420061,0.651479,0.984752,0.830215,1.121157,0.092907,-0.301643,-0.313369,-0.627826,...,1.227585,0.163779,-0.318683,0.600495,0.937776,0.311725,0.631726,-0.752176,0.671399,0.536154
3,-0.333780,0.103347,0.651479,-1.863632,-0.720298,-0.815039,-0.499274,-0.061670,-0.687324,-0.521734,...,-0.761621,0.163779,0.296763,0.600495,0.812705,1.650307,0.790804,-0.752176,0.586389,-0.515281
4,0.715614,0.878409,1.374795,0.951632,0.733308,1.414325,0.463568,-0.174865,0.199680,-0.045611,...,1.227585,1.390023,1.527656,0.600495,0.896086,1.650307,1.698485,0.780197,0.994591,0.869843


In [31]:
train_stan=pd.concat([train_con,train_cat],axis=1)
train_stan.head()

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,-0.084142,-0.133231,0.651479,1.050994,0.878668,1.193663,0.575425,-0.944591,-0.459303,-0.793434,...,0,0,0,1,0,0,0,0,1,0
1,0.563447,0.113442,-0.071836,0.156734,-0.429577,-0.815039,1.171992,-0.641228,0.466465,0.257140,...,0,0,0,1,0,0,0,0,1,0
2,0.056580,0.420061,0.651479,0.984752,0.830215,1.121157,0.092907,-0.301643,-0.313369,-0.627826,...,0,0,0,1,0,0,0,0,1,0
3,-0.333780,0.103347,0.651479,-1.863632,-0.720298,-0.815039,-0.499274,-0.061670,-0.687324,-0.521734,...,0,0,0,1,1,0,0,0,0,0
4,0.715614,0.878409,1.374795,0.951632,0.733308,1.414325,0.463568,-0.174865,0.199680,-0.045611,...,0,0,0,1,0,0,0,0,1,0


In [32]:
y=df[['SalePrice']]
y.head()                           #Target Feature

,SalePrice
0,208500
1,181500
2,223500
3,140000
4,250000


In [33]:
x=train_stan.drop('SalePrice',axis=1)         #contains all dependent feature

In [228]:
from statsmodels.api import OLS,add_constant
const=add_constant(x)
ols=OLS(y,const)
model=ols.fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.919
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     174.6
Date:                Tue, 06 Sep 2022   Prob (F-statistic):               0.00
Time:                        11:34:15   Log-Likelihood:                -16710.
No. Observations:                1460   AIC:                         3.360e+04
Df Residuals:                    1370   BIC:                         3.408e+04
Df Model:                          89                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -1.435e+15   2.26e+16     -0.064      0.949   -4.57e+16    4.29e+16
LotArea               7110.1739   1047.742      6.786      0.000    5054.821    9165.527
OverallQual           1.098e+04   1251.705      8.774      0.000    8527.020    1.34e+04
YearBuilt             8069.8028   1548.912      5.210      0.000    5031.307    1.11e+04
YearRemodAdd          5278.2881    922.310      5.723      0.000    3468.996    7087.581
BsmtFinSF1            9260.4040    842.056     10.997      0.000    7608.546    1.09e+04
TotalBsmtSF           6340.1971   1531.556      4.140      0.000    3335.747    9344.647
1stFlrSF              2.007e+04   1593.473     12.595      0.000    1.69e+04    2.32e+04
2ndFlrSF              2.868e+04   1681.063     17.061      0.000    2.54e+04     3.2e+04
BedroomAbvGr         -2714.6191    893.511     -3.038      0.002   -4467.416    -961.822
Fireplaces            1586.8448    812.910      1.952      0.051      -7.839    3181.528
GarageArea            4800.8830    909.888      5.276      0.000    3015.959    6585.807
WoodDeckSF            1532.8375    693.277      2.211      0.027     172.839    2892.836
MSZoning_C (all)     -1.299e+15   2.04e+16     -0.064      0.949   -4.14e+16    3.88e+16
MSZoning_FV          -1.299e+15   2.04e+16     -0.064      0.949   -4.14e+16    3.88e+16
MSZoning_RH          -1.299e+15   2.04e+16     -0.064      0.949   -4.14e+16    3.88e+16
MSZoning_RL          -1.299e+15   2.04e+16     -0.064      0.949   -4.14e+16    3.88e+16
MSZoning_RM          -1.299e+15   2.04e+16     -0.064      0.949   -4.14e+16    3.88e+16
LandContour_Low      -1.051e+04   4781.634     -2.198      0.028   -1.99e+04   -1131.742
LotConfig_Corner      7837.9261   3699.888      2.118      0.034     579.867    1.51e+04
LotConfig_CulDSac     1.514e+04   4232.600      3.578      0.000    6839.141    2.34e+04
LotConfig_Inside      6889.4404   3463.236      1.989      0.047      95.620    1.37e+04
Neighborhood_ClearCr -1.379e+04   5676.122     -2.430      0.015   -2.49e+04   -2658.522
Neighborhood_CollgCr -1.159e+04   3414.593     -3.393      0.001   -1.83e+04   -4887.513
Neighborhood_Crawfor  1.457e+04   4089.390      3.563      0.000    6547.139    2.26e+04
Neighborhood_Edwards -1.716e+04   3259.283     -5.265      0.000   -2.36e+04   -1.08e+04
Neighborhood_Gilbert -1.402e+04   4134.243     -3.391      0.001   -2.21e+04   -5907.771
Neighborhood_Mitchel -2.314e+04   4291.417     -5.391      0.000   -3.16e+04   -1.47e+04
Neighborhood_NAmes   -1.545e+04   2947.049     -5.244      0.000   -2.12e+04   -9673.783
Neighborhood_NWAmes  -1.553e+04   3887.051     -3.994      0.000   -2.32e+04   -7900.682
Neighborhood_NoRidge  3.047e+04   5047.089      6.037      0.000    2.06e+04    4.04e+04
Neighborhood_NridgHt  1.606e+04   4214.138      3.811    

In [223]:
A=pd.DataFrame(model.pvalues,columns=['pvals']).round(3)
A.drop('const',axis=0,inplace=True)
A.sort_values(by='pvals',ascending=False)

,pvals
Neighborhood_SWISU,0.197
LandContour_HLS,0.180
GarageQual_Gd,0.172
BsmtFinType2_GLQ,0.150
MasVnrType_Stone,0.146
...,...
Neighborhood_NAmes,0.000
Neighborhood_Mitchel,0.000
BsmtQual_Ex,0.000
RoofMatl_ClyTile,0.000


In [225]:
drop_col=A[A.pvals >0.100 ].index
print(drop_col)

Index(['FullBath', 'GarageCars', 'LotShape_IR1', 'LandContour_HLS',
       'Neighborhood_NPkVill', 'Neighborhood_SWISU', 'BldgType_Duplex',
       'HouseStyle_2.5Fin', 'MasVnrType_Stone', 'BsmtFinType1_GLQ',
       'BsmtFinType1_LwQ', 'BsmtFinType2_GLQ', 'GarageQual_Gd'],
      dtype='object')


In [226]:
x=x.drop(labels=drop_col,axis=1)

In [227]:
x.shape

(1460, 92)

In [220]:
x1=pd.DataFrame(x)

In [230]:
x1.shape         # will go with this

(1460, 105)

In [231]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x1,y,test_size=0.2,random_state=10)

In [232]:
models1= {
          "LinearRegresion":LinearRegression(),
          "DecisionTreeRegressor":DecisionTreeRegressor(),
          "RandomForestRegressor":RandomForestRegressor(),
          "AdaBoostRegressor":AdaBoostRegressor(),
          "XGBRegressor":XGBRegressor(),
          "GradientBoostingRegressor":GradientBoostingRegressor(),
          "Ridge":Ridge(),
          "Lasso":Lasso()
} 
      
for i in range(len(list(models))):
        model=list(models.values())[i]
        model.fit(xtrain,ytrain)
        
        #make predictions
        ytrain_pred=model.predict(xtrain)
        ytest_pred=model.predict(xtest)
        
        #training set performance
        model_train_error=mean_absolute_error(ytrain,ytrain_pred)
       
        #testing set performances
        model_test_error=mean_absolute_error(ytest,ytest_pred)
       
        print(list(models.keys())[i])
        
        print('Training Error :{:.4f}'.format(model_train_error))
    
        print('-'*40)
        
        print('Testing Error :{:.4f}'.format(model_test_error))
    
        print("="*40)
        print("\n")
        

LinearRegresion
Training Error :17699.3878
----------------------------------------
Testing Error :326757058566636.2500


DecisionTreeRegressor
Training Error :0.0000
----------------------------------------
Testing Error :28796.7842


RandomForestRegressor
Training Error :6697.1369
----------------------------------------
Testing Error :19988.2192


AdaBoostRegressor
Training Error :22625.0196
----------------------------------------
Testing Error :26970.6906


XGBRegressor
Training Error :1147.5714
----------------------------------------
Testing Error :18160.3999


GradientBoostingRegressor
Training Error :10972.2082
----------------------------------------
Testing Error :18109.2728


Ridge
Training Error :15234.6756
----------------------------------------
Testing Error :16440.9586


Lasso
Training Error :14326.4490
----------------------------------------
Testing Error :15901.3788




In [233]:
#Hyper Parameters
#Decisiontree
params1 = {"max_depth": [2,4,8,12,16,20,25,30,40,50],
             "max_features": [2,4,8,'auto'],
             "min_samples_split" :[2,4,8],
             "min_samples_leaf": [2,4,8]}
#RandomForestRegressor
params2 = {"max_depth": [2,4,8,12,16,20,25,30,40,50],
             "max_features": [2,4,8,'auto'],
             "n_estimators": [50,100,200,500],
             "min_samples_split" :[2,4,8],
             "min_samples_leaf": [2,4,8]}
#AdaBoostRegressor
params3={ 'base_estimator' :[DecisionTreeRegressor(max_depth=2)],
             "n_estimators": [50,100,200,500],
             'learning_rate':[0.5,0.9,1.0],
             'loss':['linear','square','exponential']}
#XGBRegressor
params4={"n_estimators": [50,100,200,500],
            'max_depth':[2,4,8,12,16,20,25,30,40,50], 
            'eta'      :[0.01,0.05,0.1,0.5,0.9,1],
            'subsample':[0.8,1],
            'colsample_bytree':[0.8,1]}
#GradientBoostingRegressor
params5 = {  'learning_rate': [0.5,0.9,1.0],
              'subsample'   : [0.8,1],
              "n_estimators": [50,100,200,500],
              'max_depth'   : list(range(2,50,10))}
#Ridge 
params6= {'alpha': [200, 230, 250,270,290,300,500] ,
           "fit_intercept": [True, False]}
#Lasso
params7= {'alpha': [1,0.1,0.01,0.001,0.0001,0],
           "fit_intercept": [True, False]}

In [234]:
#models list for hyper parameter tuning
randomcv_models = [("DecisionTreeRegressor",DecisionTreeRegressor(),params1),
                   ('RandomForestRegressor',RandomForestRegressor(),params2),
                   ('AdaBoostRegressor',AdaBoostRegressor(),params3),
                   ("XGBRegressor",XGBRegressor(),params4),
                   ("Ridge",Ridge(),params6),
                   ("Lasso",Lasso(),params7)
]

randomcv_models1 = [("GradientBoostingRegressor",GradientBoostingRegressor(),params5)
]

In [235]:
#training the model
from sklearn.model_selection import RandomizedSearchCV

model_param={}
for name,model,params in randomcv_models:
    random = RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=4,verbose=2,n_jobs=-1)
    
    random.fit(xtrain,ytrain)
    model_param[name] = random.best_params_
    
for model_name in model_param:
    print(f"---------------------Best Params for =={model_name}-----------------------")
    print(model_param[model_name])
    print("\n")

Fitting 4 folds for each of 100 candidates, totalling 400 fits
Fitting 4 folds for each of 100 candidates, totalling 400 fits
Fitting 4 folds for each of 36 candidates, totalling 144 fits
Fitting 4 folds for each of 100 candidates, totalling 400 fits
Fitting 4 folds for each of 14 candidates, totalling 56 fits
Fitting 4 folds for each of 12 candidates, totalling 48 fits
---------------------Best Params for ==DecisionTreeRegressor-----------------------


{'min_samples_split': 8, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 20}
---------------------Best Params for ==RandomForestRegressor-----------------------


{'n_estimators': 500, 'min_samples_split': 4, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 16}
---------------------Best Params for ==AdaBoostRegressor-----------------------


{'n_estimators': 50, 'loss': 'exponential', 'learning_rate': 0.5, 'base_estimator': DecisionTreeRegressor(max_depth=2)}
---------------------Best Params for ==XGBRegressor---

In [236]:
from sklearn.model_selection import RandomizedSearchCV

model_param={}
for name,model,params in randomcv_models1:
    random = RandomizedSearchCV(estimator=model,param_distributions=params,n_iter=100,cv=4,verbose=2,n_jobs=-1)
    
    random.fit(xtrain,ytrain)
    model_param[name] = random.best_params_
    
for model_name in model_param:
    print(f"---------------------Best Params for =={model_name}-----------------------")
    print(model_param[model_name])
    print("\n")
    
    

Fitting 4 folds for each of 100 candidates, totalling 400 fits
---------------------Best Params for ==GradientBoostingRegressor-----------------------


{'subsample': 0.8, 'n_estimators': 50, 'max_depth': 42, 'learning_rate': 0.5}


In [386]:
#Regularised Models
models2= {"RandomForestRegressor":RandomForestRegressor(n_estimators=500,min_samples_split=2,
                                                       min_samples_leaf=2,max_features='auto',max_depth=50),
          
         "AdaBoostRegressor":AdaBoostRegressor(n_estimators=50,loss='exponential',
                                               learning_rate=0.5,base_estimator=DecisionTreeRegressor(max_depth=2)),
          
         "XGBRegressor":XGBRegressor(subsample=0.8,n_estimators=500,max_depth=20,eta=0.01,colsample_bytree=0.8),
          
         "Ridge":Ridge(solver='sag',fit_intercept=True, alpha=200),
          
         "Lasso":Lasso(fit_intercept=False, alpha=1),
          
         "GradientBoostingRegressor":GradientBoostingRegressor(subsample=0.8, n_estimators=50, max_depth=32, learning_rate=0.5)
} 

for i in range(len(list(models))):
        model=list(models.values())[i]
        model.fit(xtrain,ytrain)
        
        #make predictions
        ytrain_pred=model.predict(xtrain)
        ytest_pred=model.predict(xtest)
        
        model_train_error=mean_absolute_error(ytrain,ytrain_pred)
        model_test_error=mean_absolute_error(ytest,ytest_pred)
        print(list(models.keys())[i])
        
        print('Training Error :{:.4f}'.format(model_train_error))
    
        print('-'*40)
        
        print('Testing Error :{:.4f}'.format(model_test_error))
    
        print("="*40)
        print("\n")
        

RandomForestRegressor
Training Error :7808.8578
----------------------------------------
Testing Error :19622.9342


AdaBoostRegressor
Training Error :26798.2246
----------------------------------------
Testing Error :31384.5083


XGBRegressor
Training Error :2454.5341
----------------------------------------
Testing Error :17332.4985


Ridge
Training Error :18460.6098
----------------------------------------
Testing Error :19233.1301


Lasso
Training Error :14326.4899
----------------------------------------
Testing Error :15924.3349


GradientBoostingRegressor
Training Error :0.0001
----------------------------------------
Testing Error :22896.3983




In [ ]:
#Lasso is best fitting 

# Using GridSearchCV

In [384]:
#Ridge 
params8= {'alpha': [1,1.5,0.1,0.5,0.01,0.05,0.001,0.0001,100,200, 230, 250,270,290,300,500] ,
           "fit_intercept": [True, False]}
#Lasso
params9= {'alpha': [1.5,0.5,0.05,1,0.1,0.01,0.001,0.0001,100,200, 230, 250,270,290,300,500],
           "fit_intercept": [True, False]}

randomcv_models2 = [("Ridge",Ridge(),params8),
                   ("Lasso",Lasso(),params9)]

In [385]:
#training the model
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV

model_param={}
for name,model,params in randomcv_models2:
    random = GridSearchCV(estimator=model,param_grid=params,scoring='neg_mean_absolute_error',cv=5,verbose=2,n_jobs=-1)
    
    random.fit(xtrain,ytrain)
    model_param[name] = random.best_params_
    
for model_name in model_param:
    print(f"---------------------Best Params for =={model_name}-----------------------")
    print(model_param[model_name])
    print("\n")

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Fitting 5 folds for each of 32 candidates, totalling 160 fits
---------------------Best Params for ==Ridge-----------------------
{'alpha': 0.1, 'fit_intercept': True}


---------------------Best Params for ==Lasso-----------------------
{'alpha': 100, 'fit_intercept': True}




In [446]:
#Regularised Models
models3= {
         "Ridge":Ridge(solver='sag',fit_intercept=True, alpha=0.1),
          
         "Lasso":Lasso(fit_intercept=False, alpha=100),
         } 

for i in range(len(list(models))):
        model=list(models.values())[i]
        model.fit(xtrain,ytrain)
        
        #make predictions
        ytrain_pred=model.predict(xtrain)
        ytest_pred=model.predict(xtest)
        
        model_train_error=mean_absolute_error(ytrain,ytrain_pred)
        model_test_error=mean_absolute_error(ytest,ytest_pred)
        print(list(models.keys())[i])
        
        print('Training Error :{:.4f}'.format(model_train_error))
    
        print('-'*40)
        
        print('Testing Error :{:.4f}'.format(model_test_error))
    
        print("="*40)
        print("\n")
        

Ridge
Training Error :14946.6809
----------------------------------------
Testing Error :16386.5100


Lasso
Training Error :15501.6651
----------------------------------------
Testing Error :16259.4885




In [391]:
x1.shape

(1460, 105)

# Creating a NN

In [388]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [412]:
nn = Sequential()
nn.add(Dense(600,activation="relu",input_dim=(105)))
nn.add(Dense(1500,activation="relu"))
nn.add(Dense(1))

In [443]:
nn.compile(optimizer="adam",loss="mean_absolute_error")
nn.fit(xtrain,ytrain,epochs=800,initial_epoch=300)

Epoch 301/800
37/37 [==============================] - 1s 20ms/step - loss: 11722.6660
Epoch 302/800
37/37 [==============================] - 1s 24ms/step - loss: 11698.3643
Epoch 303/800
37/37 [==============================] - 1s 35ms/step - loss: 11666.5000
Epoch 304/800
37/37 [==============================] - 1s 24ms/step - loss: 11590.8398
Epoch 305/800
37/37 [==============================] - 1s 31ms/step - loss: 11583.3770
Epoch 306/800
37/37 [==============================] - 1s 28ms/step - loss: 11600.1338
Epoch 307/800
37/37 [==============================] - 1s 23ms/step - loss: 11600.4902
Epoch 308/800
37/37 [==============================] - 1s 22ms/step - loss: 11610.6240
Epoch 309/800
37/37 [==============================] - 1s 23ms/step - loss: 11566.7812
Epoch 310/800
37/37 [==============================] - 1s 21ms/step - loss: 11604.9307
Epoch 311/800
37/37 [==============================] - 1s 20ms/step - loss: 11666.7217
Epoch 312/800
37/37 [======================

In [444]:
pred_tr = nn.predict(xtrain)
pred_ts = nn.predict(xtest)

tr_err = mean_absolute_error(ytrain,pred_tr)
ts_err = mean_absolute_error(ytest,pred_ts)
print("Training Error",tr_err)
print('-'*40)
print("Testing Error",ts_err)

Training Error 9313.682811162244
----------------------------------------
Testing Error 15365.461111408391


In [380]:
list(models.values())[4]

Lasso(alpha=1, fit_intercept=False)

# Getting prediction on test data 

In [320]:
test=pd.read_csv('C:/Users/cmscomputers/Desktop/datasets/House_testing_set.csv')

In [321]:
test1=test.drop(labels=['PoolQC','MiscFeature','Alley','Fence'],axis=1)

In [322]:
test1=test1.drop(labels=E,axis=1)

In [323]:
for i in test1:
    if(test1[i].dtypes=='object'):
        c=test1[i].mode()[0]
        test1[i]=test1[i].fillna(c)         #this iteration replaces missing values from categorical data
    else:
        c=test1[i].mean()
        test1[i]=test1[i].fillna(c) 

In [324]:
test1.isna().sum().sum()

0

p.remove('SalePrice')

In [326]:
test1=test1[p]

In [327]:
test_cat=pd.get_dummies(test1[cat])
test_cat.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,1,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


con.remove('SalePrice')

In [329]:
test_con=pd.DataFrame(ss.fit_transform(test1[con]),columns=con)
test_con.head()

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF
0,0.555587,0.363929,-0.751101,-0.340945,-1.072885,-0.570108,0.063295,-0.650619,-0.370808,-0.654561,...,-1.028720,-0.751040,-1.029543,-0.918335,-0.898055,-0.650488,-0.988013,1.185945,0.366678,-0.701628
1,0.604239,0.897861,-0.054877,-0.439695,-1.214908,0.041273,1.063392,-0.339378,0.639144,0.433298,...,-1.028720,1.237648,0.175997,-0.255371,-0.898055,-0.767194,-0.988013,-0.741213,2.347867,-0.178826
2,0.263676,0.809646,-0.751101,0.844059,0.678742,-0.570108,0.773254,-0.954994,-0.266876,-0.574165,...,0.773083,1.237648,0.175997,-0.255371,0.647066,0.749983,0.301623,0.042559,0.930495,-0.207871
3,0.458284,0.032064,-0.054877,0.876976,0.678742,-0.456889,0.357829,-0.527038,-0.271395,-0.579190,...,0.773083,1.237648,0.175997,0.407593,0.647066,0.788885,0.301623,-0.012766,2.089451,-0.178826
4,-1.244533,-0.971808,1.337571,0.679475,0.394694,-0.570108,-0.387298,1.058917,0.528434,0.310192,...,0.773083,-0.751040,-1.029543,-0.918335,-0.898055,0.555473,0.301623,0.153210,-0.729632,0.489198


In [330]:
test_stan=pd.concat([test_con,test_cat],axis=1)
test_stan.head()

,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.555587,0.363929,-0.751101,-0.340945,-1.072885,-0.570108,0.063295,-0.650619,-0.370808,-0.654561,...,0,0,0,1,0,0,0,0,1,0
1,0.604239,0.897861,-0.054877,-0.439695,-1.214908,0.041273,1.063392,-0.339378,0.639144,0.433298,...,0,0,0,1,0,0,0,0,1,0
2,0.263676,0.809646,-0.751101,0.844059,0.678742,-0.570108,0.773254,-0.954994,-0.266876,-0.574165,...,0,0,0,1,0,0,0,0,1,0
3,0.458284,0.032064,-0.054877,0.876976,0.678742,-0.456889,0.357829,-0.527038,-0.271395,-0.579190,...,0,0,0,1,0,0,0,0,1,0
4,-1.244533,-0.971808,1.337571,0.679475,0.394694,-0.570108,-0.387298,1.058917,0.528434,0.310192,...,0,0,0,1,0,0,0,0,1,0


In [331]:
r=x1.columns

In [332]:
e=['HouseStyle_2.5Fin', 'RoofMatl_ClyTile', 'RoofMatl_Membran', 'RoofMatl_Metal', 'RoofMatl_Roll', 'Exterior1st_ImStucc', 'Heating_OthW', 'GarageQual_Ex']

In [333]:
test_stan[e]=0

In [334]:
Test=test_stan[r]

In [335]:
Test.shape

(1459, 105)

In [447]:
for i in range(len(list(models3))):
        model=list(models3.values())[0]
        model.fit(xtrain,ytrain)
        
        #make predictions
        pred1=model.predict(Test)                  #Ridge is selected

In [448]:
pred2=nn.predict(Test)          #prediction from NN

In [451]:
pred1

array([[123491.98625238],
       [163104.95067154],
       [188925.75616493],
       ...,
       [172955.81620866],
       [108237.24282809],
       [223682.20499807]])

In [452]:
pred2

array([[128155.625],
       [159497.6  ],
       [194181.12 ],
       ...,
       [171546.97 ],
       [130588.04 ],
       [220665.   ]], dtype=float32)

In [453]:
test['SalePrice']=pred1

In [457]:
Prediction=test[['Id','SalePrice']]

Prediction.to_csv('House_Price_predicted2.csv',index=False)

In [459]:
pd.read_csv('House_Price_predicted2.csv')

,Id,SalePrice
0,1461,123491.986252
1,1462,163104.950672
2,1463,188925.756165
3,1464,195086.483147
4,1465,203872.548434
...,...,...
1454,2915,88552.195669
1455,2916,92741.291743
1456,2917,172955.816209
1457,2918,108237.242828
